In [1]:
from lltk.model.charnet import *
# bokeh_uses_notebook()

In [2]:
URL_ODYSSEY_DYN='https://docs.google.com/spreadsheets/d/e/2PACX-1vS19Gbv7qN-OTpb7yyNTrxLEIoBOVfDnXemxweLwYeU4FLo1zH3efysgQrNH70mkM90_Rv3AJjHD-EC/pub?gid=2343770&single=true&output=csv'
URL_AETHIOPICA_DYN='https://docs.google.com/spreadsheets/d/e/2PACX-1vSITB0rsQH9TKUwfBjGdCBlkk7PyDHlyNQtnnqoWwIWyvWfAai2qUbbdnJPwMr66SHo5NC86b8-_X-c/pub?gid=695648537&single=true&output=csv'
URL_MOBYDICK = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vT46sYCUH-hJAcKoSqwSYMsJ-zOdBYYk7shl-hPoB55C12J_GHDN6ar03W_zQV6ueEibmHXmR5lPL1m/pub?gid=1995777382&single=true&output=csv'

In [3]:
cnet=CharacterNetwork()
cnet.parse_dynamic(URL_ODYSSEY_DYN)
cnet.df2nx(i_max=10)
for a,b,d in cnet.g.edges(data=True):
    if d['last_edge']:
        print(a,b,d)
for n,dat in cnet.g.nodes(data=True):
    if cnet.g.nodes[n]['last_node']:
       print(n,'!')
    # print(n,dat)

Geolocating settings: 100%|██████████| 14/14 [00:01<00:00,  8.11it/s]Penelope Antinous {'weight': 1, 'index': 10, 'i': 10, 't': 1, 'source': 'Penelope', 'target': 'Antinous', 'name': 'Book I', 'setting': 'Ithaca', 'storylevel': '1', 'lat': 42.4396039, 'lon': -76.4968019, 'last_edge': True}
Penelope !



In [4]:
set(cnet.df_nodes.columns)&set(cnet.df_edges.columns)

{'name'}

In [5]:
cnet.settings_latlon()

{'Scheria': [52.09024855, 4.435438387970711],
 'Aeaea': [38.28958405, -122.46132401796646],
 'Aeolia': [53.299365550000005, -2.241047249999998],
 'Ogygia': [34.836659499999996, 24.083214861544448],
 'Olympus': [34.9366134, 32.8649502],
 'Hades': [50.1765988, 19.746864614118905],
 'Eridanus': [-31.4540181, -64.1761992],
 'Sparta': [37.08114525, 22.42486178272022],
 'Pylos': [-22.89547905, -43.19603118133239],
 'Ithaca': [42.4396039, -76.4968019]}

In [6]:
l=[1,0]
l[0]

1

In [11]:
cnet.df2nx()
cnet.g.nodes(data=True)['Odysseus']

{'last_node': False,
 'index': 245,
 'id': 'Odysseus',
 'name': 'Odysseus',
 'degree': 63,
 'betweenness_centrality': 0.5361580086580087,
 'i': 245,
 't': 24,
 'source': 'Odysseus',
 'target': 'Laertes',
 'setting': 'Hades, Ithaca',
 'storylevel': '1',
 'lat': 42.4396039,
 'lon': -76.4968019}

In [8]:
stop

NameError: name 'stop' is not defined

In [20]:
def make_gif_from_folder(folder,ofn=None,fps=5):
    import imageio
    from IPython.display import HTML

    images = []
    # for fn in tqdm(sorted(os.listdir(folder)),desc='Building gif from images'):
    for fn in sorted(os.listdir(folder)):#,desc='Building gif from images'):
        if fn.endswith('.png'):
            with open(os.path.join(folder,fn),'rb') as f:
                images.append(imageio.imread(f))

    if not ofn:
        if folder.endswith(os.path.sep): folder=folder[:-1]
        ofn=folder+'.gif'        
    imageio.mimsave(ofn, images, duration = 1/fps)
    return HTML(f'''<img src="{ofn}" />''')

    # # show gif?
    # from IPython.core.interactiveshell import InteractiveShell
    # InteractiveShell.ast_node_interactivity = "all"
    # from IPython.display import Image
    # from pathlib import Path
    # gifPath = Path(ofn_gif)
    # if os.path.exists(gifPath):
    #     # Display GIF in Jupyter, CoLab, IPython
    #     with open(gifPath,'rb') as f:
    #         return Image(data=f.read(), format='png')


In [22]:
# !pip install geopy pyproj

In [7]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent='myapplication')
location = geolocator.geocode(["Chemmis",'Chicago'])
location

In [33]:
[geolocator.geocode('werwfwefewfwefgew')]

[None]

In [15]:
def shownet(url,bad_nodes=[],**y):
    cnet = CharacterNetwork()
    cnet.parse_dynamic(url)
    cnet.df2nx()
    cnet.gen_stats()
    for bn in bad_nodes: cnet.g.remove_node(bn)
    pos=nx.circular_layout(cnet.g)
    return draw_bokeh(cnet.g,pos=pos,**y)#,spring_scale=20,iterations=100)

In [16]:
def do_shownets(obj,*args,**kwargs):
    g,obj_kwds=obj
    draw_bokeh(
        g,
        **obj_kwds,
        **kwargs
    )

def shownets(url,bad_nodes=[],odir='graph_imgs',size_by='degree',fps=5,num_proc=4,layout='force',**y):
    import shutil
    if os.path.exists(odir): shutil.rmtree(odir)
    if not os.path.exists(odir): os.makedirs(odir)

    # overall pos
    cnet = CharacterNetwork()
    cnet.parse_dynamic(url,bad_nodes=set(bad_nodes))
    cnet.df2nx(t_min=None,t_max=None)

    if layout:
        pos=layout_graph(cnet.g,**y)
    else:
        pos=None

    timesteps=sorted(list(set(cnet.df_edges.t)))

    size_max_val = cnet.df_nodes[size_by].max() if size_by else None

    # for t in tqdm(timesteps,f'Saving graph images to {odir}'):
    objs = []


    # for i,row in tqdm(cnet.df_edges.iterrows(),total=len(cnet.df_edges),desc=f'Generating graphs over time'):
    for i,row in cnet.df_edges.iterrows():#,total=len(cnet.df_edges),desc=f'Generating graphs over time'):
        # cnet.df2nx(t_max=t)
        
        title=f'Interation #{str(i+1).zfill(4)}: {row.source} to {row.target}'
        if row.get('name'): title+=f' ({row["name"]})'
        if row.get('setting'): title+=f', in {row.setting}'
        if row.get('id'): title+=f'[t={row.id}]'

        ofn=os.path.join(odir,f'graph.t_{str(i).zfill(4)}.png')
        obj_kwds = dict(
            save_to=ofn,
            title=title
        )
        objs += [(cnet.df2nx(i_max=i).copy(),obj_kwds)]

    pmap(
        do_shownets,
        objs,
        kwargs=dict(
            show_plot=False,
            pos=pos,
            size_by=size_by,
            size_max_val=size_max_val,
            color_line_by='last_edge',
            color_by='last_node',
            **y
        ),
        desc=f'Saving graph images to {odir}',
        num_proc=num_proc,
        progress=False
    )

    # make gif?
    return make_gif_from_folder(odir,fps=fps)
    

In [17]:
shownets(
    URL_ODYSSEY_DYN,
    scalingRatio=0.035,
    # min_degree=2,
    bad_nodes=['Ithacan assembly','Phaecian assembly','Suitors'],
    odir='graphs_Odyssey',
)

In [18]:
shownets(
    URL_AETHIOPICA_DYN,
    scalingRatio=0.05,
    odir='graphs_Aethiopica',
)

In [19]:
shownets(
    URL_MOBYDICK,
    size_by='degree',
    scalingRatio=0.05,
    iterations=10000,
    bad_nodes=['All','HypotheticalHarpooner','Sailors'],
    odir='graphs_MobyDick'
    # min_degree=2
)